#### Importing the Libraries

In [2]:
import pandas as pd 
import numpy as np
from tensorflow.keras.layers import Dense, LSTM, Input 
from tensorflow.keras.models import Model 

In [3]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000

data_path = 'dataset/npi.txt'

#### Vectorizing the Data 

In [3]:
# vectorize the data 
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, 'r', encoding='utf-8') as f :
    lines = f.read().split('\n')
for line in lines[:min(len(lines)-1, num_samples)]:
    input_text, target_text, _ = line.split('\t')
    # we use 'tab' as the 'start sequence' character 
    # for the target and '\n' as 'end sequence' characters 
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)

    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [4]:
print(input_texts)
print(target_texts)
print(input_characters)
print(target_characters)

['Who?', 'Hide.', 'Hide.', 'Stay.', 'Hello!', 'Smile.', 'Attack!', 'Go slow.', "I'm Tom.", 'Find Tom.', 'I am Tom.', 'I danced.', 'Who am I?', 'Can I eat?', 'He is old.', 'I am okay.', 'I am sick.', 'I am sure.', 'I am tall.', 'I am well.', "I'm crazy.", "I'm crazy.", "I'm sorry.", 'Is it bad?', 'Is it bad?', 'May I eat?', 'Who is he?', 'Good night.', 'He is lazy.', 'He is nice.', 'He is nice.', 'He is poor.', 'He is sick.', 'He is sick.', 'How lovely!', 'I am a boy.', 'I am a man.', 'I am happy.', 'I am human.', 'I am human.', 'I am short.', 'I am smart.', 'I eat fish.', 'I eat here.', 'I eat meat.', 'I eat pork.', 'I eat pork.', 'I eat rice.', 'I feel old.', 'I like you.', 'I love you.', 'I miss you.', 'I was hurt.', "I'm chubby.", "I'm chubby.", 'Love hurts.', 'Turn it up.', 'We are men.', 'Who is Tom?', "Beer's good.", "Beer's good.", "Don't worry.", "Don't worry.", 'Have a beer.', 'He is alone.', 'He is alone.', 'He is eight.', 'He is happy.', 'He is happy.', 'Here we are!', 'Here

In [5]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

for line in lines[: min(num_samples, len(lines)-1)]:
    input_text, target_text, _ =  line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)

    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)

    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [6]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
print("The input characters :\n", input_characters)
print("The target characters :\n", target_characters)

The input characters :
 [' ', '!', '"', "'", ',', '-', '.', '0', '1', '2', '3', '4', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '€']
The target characters :
 ['\t', '\n', ' ', '!', '"', ',', '-', '0', '2', '3', ':', '?', 'ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'े', 'ै', 'ो', 'ौ', '्', '।', '०', '१', '२', '३', '४', '५', '९', '\u200b', '\u200d']


In [7]:
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
print("Number of unique character Encoded tokens: ", num_encoder_tokens)
print("Number of unique character Decoder tokens: ", num_decoder_tokens)

Number of unique character Encoded tokens:  67
Number of unique character Decoder tokens:  80


In [8]:
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])
print("maximum encoder input sequence length : ", max_encoder_seq_length)
print("maximum decoder input sequence length : ", max_decoder_seq_length)

maximum encoder input sequence length :  123
maximum decoder input sequence length :  104


In [9]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)]
)
print("input character index:", input_token_index)
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)]
)
print("target character index:", target_token_index)

input character index: {' ': 0, '!': 1, '"': 2, "'": 3, ',': 4, '-': 5, '.': 6, '0': 7, '1': 8, '2': 9, '3': 10, '4': 11, '9': 12, ':': 13, '?': 14, 'A': 15, 'B': 16, 'C': 17, 'D': 18, 'E': 19, 'F': 20, 'G': 21, 'H': 22, 'I': 23, 'J': 24, 'K': 25, 'L': 26, 'M': 27, 'N': 28, 'O': 29, 'P': 30, 'Q': 31, 'R': 32, 'S': 33, 'T': 34, 'U': 35, 'V': 36, 'W': 37, 'Y': 38, 'Z': 39, 'a': 40, 'b': 41, 'c': 42, 'd': 43, 'e': 44, 'f': 45, 'g': 46, 'h': 47, 'i': 48, 'j': 49, 'k': 50, 'l': 51, 'm': 52, 'n': 53, 'o': 54, 'p': 55, 'q': 56, 'r': 57, 's': 58, 't': 59, 'u': 60, 'v': 61, 'w': 62, 'x': 63, 'y': 64, 'z': 65, '€': 66}
target character index: {'\t': 0, '\n': 1, ' ': 2, '!': 3, '"': 4, ',': 5, '-': 6, '0': 7, '2': 8, '3': 9, ':': 10, '?': 11, 'ँ': 12, 'ं': 13, 'ः': 14, 'अ': 15, 'आ': 16, 'इ': 17, 'ई': 18, 'उ': 19, 'ऊ': 20, 'ऋ': 21, 'ए': 22, 'ऐ': 23, 'ओ': 24, 'औ': 25, 'क': 26, 'ख': 27, 'ग': 28, 'घ': 29, 'ङ': 30, 'च': 31, 'छ': 32, 'ज': 33, 'झ': 34, 'ञ': 35, 'ट': 36, 'ठ': 37, 'ड': 38, 'ढ': 39, 'ण': 4

In [10]:
print(f"size of input token index dictionary : {len(input_token_index)}")
print(f"size of target token index dictionary : {len(target_token_index)}")


size of input token index dictionary : 67
size of target token index dictionary : 80


In [11]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32'
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32'
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32'
)

In [12]:
print("encoder input data ", encoder_input_data.shape)
print("decoder input data ", decoder_input_data.shape)
print("decoder target data ", decoder_target_data.shape)

encoder input data  (2821, 123, 67)
decoder input data  (2821, 104, 80)
decoder target data  (2821, 104, 80)


In [13]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1

    for t, char in enumerate(target_text):
        # decoder target data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1

    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1
    decoder_target_data[i, t:, target_token_index[' ']] = 1

In [14]:
decoder_input_data

array([[[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       ...,

       [[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0.

In [15]:
encoder_input_data[0].shape

(123, 67)

In [16]:
# Input layers are defined to sequenc and process it. 
# the shape of the layer for the variable length sequences is none 
# and in each time step, the size of the num_encoder_tokens are one hot vectorized. 

encoder_inputs = Input(shape=(None, num_encoder_tokens))
print(encoder_inputs)

<KerasTensor shape=(None, None, 67), dtype=float32, sparse=False, ragged=False, name=keras_tensor>


In [17]:
# I 'm creating a LSTM layer with latent_dim units and return state tells the lstm to give ouput 
# encoder_outputs : full_sequence 
# state_h = hidden state ( think at is as what lstm layer is going to learn in current state)
# state_c = cell state ( thinks at is as what lstm layer learned in that input row)

encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

In [18]:
# I'm packing the two encoder states into a list, which will be passed to the decoder as its inital state. 

encoder_states = [state_h, state_c]

In [19]:
# now defining the decoder setup 
# decoder layer with input as variable lenght input = None, and one hot vectors as num_decoder_tokens

decoder_inputs = Input(shape=(None, num_decoder_tokens))

In [20]:
# return_sequences = True , output a sequence (not just a final output)
# return_state = True, return hidden and cell states for inference model
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)

In [21]:
# decoder first state (inital_state) is the encoder final state 
# decoder_outputs : full sequence of decoder outputs (used for training)
# _, _ : we ignore the hidden and cell states during training. 

decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

In [22]:
# after creating Lstm layer I'm creating a dense (fully connected) layer. 
# It turns the LSTM output into a probabiility distribution

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [23]:
# Model creates the full training model. 
# this model takes both encoder input and decoder input
# This model i'll train with input/output sentence pairs. 

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [24]:
# i will train model with optimizer = 'rmsprop', loss='categorical_crossentropy')

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, 
          batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 260ms/step - loss: 1.3799 - val_loss: 2.2131
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 9s 250ms/step - loss: 1.1008 - val_loss: 1.9878
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 221ms/step - loss: 0.9649 - val_loss: 1.8857
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 211ms/step - loss: 0.9217 - val_loss: 1.5813
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 211ms/step - loss: 0.9094 - val_loss: 1.5211
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 218ms/step - loss: 0.8675 - val_loss: 1.4626
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 215ms/step - loss: 0.8664 - val_loss: 1.5663
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 217ms/step - loss: 0.8305 - val_loss: 1.5086
Epoch 9/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 214ms/step - loss: 0.8394 - val_loss: 1.4238
Epoch 10/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 226ms/step - loss: 0.8069 - val_loss: 1.3791
Epoch 11/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 216ms/step - loss: 0.8144 - val_loss: 1.4334
Epoch 12/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 

In [25]:
# encoder_model takes the encoder_inputs as input_sentence, 
# take output the final hidden state and cell state (encoder_states) from the encoder LSTM
# These states summarize the input sentence. 
encoder_model = Model(encoder_inputs, encoder_states)

In [26]:
# defining the decoder inital states , they are current state and cell state.
# current state only predict the current meaning 
# cell state predicts the actual meaning given in the input context. 

decoder_state_input_h = Input(shape=(latent_dim, ))
decoder_state_input_c = Input(shape=(latent_dim, ))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [27]:
# thinks of its as decoder are like a person speaking, 
# It needs two things (decoder_input = a word)
# and initial states are (hidden states h and c)
# h = current state and c = cell state

# it then produces decoder outputs = the next guess words 
# state_h , state_c = updated memory for the next step

# keep the updated memory (h, c) for the next word

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states=[state_h, state_c]

In [28]:
# it convert decoder's raw output into word probabilities (softmax)

decoder_outputs = decoder_dense(decoder_outputs)

In [29]:
# This builds the final decoder model that, 
# Takes current word + memory 
# returns: next word + updated memory 

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, 
    [decoder_outputs] + decoder_states
)

In [30]:
# Now words will be generated. and step wise translations. 

def decode_sequence(input_seq):

    # The encoder listens to the whole input sentence and passes its memory to the decoder. 
    state_values = encoder_model.predict(input_seq)

    # start the decoder with the special "start" symbol ('\t') it means begin speaking now
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1

    # create an empty box to store the translated sentence 
    # tell decoder to keep going untill end of the word comes. 
    stop_condition = False
    decoded_sentence = ""

    # feed current word + memory to decoder -> decoder predicts nextword and updated memory
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + state_values)

        # choose the most likely word (argmax)
        # Turn it from index -> real word 
        # add it to the growing translated sentence. 
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        reverse_target_token_index = {i: char for char, i in target_token_index.items()}
        sampled_char = reverse_target_token_index[sampled_token_index]
        decoded_sentence += sampled_char


        # stop if we see the special end symbol('\n) of if sentence gets to long
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True 

        # make the choosen word the next input for decoder. 
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1
        
        state_value = [h, c]
        # update the decoder's memory so it remembers context

        return decoded_sentence

In [31]:
for seq_index in range(100):

    # take one sequence part of the training set 
    # for trying out decoding 
    input_seq = encoder_input_data[seq_index + 1: seq_index + 2 ]
    decoded_sentence = decode_sequence(input_seq)

    print("-" * 40)
    print("Input Sentence: ", input_texts[seq_index])
    print("Deocded sequence: ", decoded_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
----------------------------------------
Input Sentence:  Who?
Deocded sequence:  ट
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
----------------------------------------
Input Sentence:  Hide.
Deocded sequence:  ट
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
----------------------------------------
Input Sentence:  Hide.
Deocded sequence:  ट
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
----------------------------------------
Input Sentence:  Stay.
Deocded sequence:  ट
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
----------------------------------------
Input Sentence:  Hello!
Deocded sequence:  ट
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
----------------------------------------
Input Sentence:  Smile.
Deocded sequence:  ट
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50m

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        reverse_target_char_index = {i: char for char, i in target_token_index.items()}
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━

In [35]:
for seq_index in range(100):

    # take one sequence part of the training set 
    # for trying out decoding 
    input_seq = encoder_input_data[seq_index + 1: seq_index + 2 ]
    decoded_sentence = decode_sequence(input_seq)

    print("-" * 40)
    print("Input Sentence: ", input_texts[seq_index])
    print("Deocded sequence: ", decoded_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━